# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/hexgrad/Kokoro-82M',
 '/openbmb/MiniCPM-o-2_6',
 '/microsoft/phi-4',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/MiniMaxAI/MiniMax-Text-01',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/stabilityai/stable-point-aware-3d',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/doc

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
hexgra

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nhexgrad/Kokoro-82M\nUpdated\nabout 2 hours ago\n•\n20.6k\n•\n1.87k\nopenbmb/MiniCPM-o-2_6\nUpdated\nabout 18 hours ago\n•\n7.05k\n•\n537\nmicrosoft/phi-4\nUpdated\n9 days ago\n•\n100k\n•\n1.41k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated\n5 days ago\n•\n4.89k\n•\n446\nMiniMaxAI/MiniMax-Text-01\nUpdated\n1 day ago\n•\n1.53k\n•\n371\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.05k\n❤️\nKokoro TTS\nNow in 5 languages!\nRunning\non\nZ

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}]}


# Hugging Face Brochure

## **About Us**
**Hugging Face** is an innovative AI community that's building the future of machine learning. We provide an open platform for collaboration, where individuals and organizations can create, discover, and share state-of-the-art machine learning models, datasets, and applications. With a commitment to open-source, we're redefining how machine learning solutions are developed and shared.

## **What We Offer**
- **Models:** Explore and access over 400k machine learning models, designed for various applications.
- **Datasets:** Browse through 100k+ datasets for various ML tasks, including text, audio, and visual data.
- **Spaces:** Run and showcase live applications with ease. Share your work with the global ML community.
- **Enterprise Solutions:** Tailored services with enterprise-grade security and dedicated support, starting at just $20/user/month.

## **Who Uses Hugging Face?**
More than **50,000 organizations** worldwide trust Hugging Face, including renowned names such as:
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Meta**
- **Intel**
  
Join the thriving ecosystem where industry leaders collaborate to push the boundaries of AI.

## **Company Culture**
At Hugging Face, we believe in cultivating a vibrant and inclusive AI community. Our emphasis is on collaboration, sharing knowledge, and building an open-source future. We strive for a work environment that encourages creativity, innovation, and continuous learning. 

### **Join Our Team**
We are actively seeking talented individuals who are passionate about AI and machine learning. Explore our current openings and become part of our journey to empower the ML community!

## **Get Involved**
Whether you are a researcher, developer, or business professional, you can play a significant role in shaping the future of AI. 
- **Log In** or **Sign Up** to start collaborating!
- Contribute to our open-source projects on GitHub.
- Follow us on social media for the latest updates and insights.

**Embrace the future with Hugging Face!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**

At Hugging Face, we are at the forefront of machine learning collaboration, providing a platform where enthusiasts, developers, and researchers can create, discover, and collaborate on machine learning (ML) models, datasets, and applications. With more than 50,000 organizations utilizing our services, we are shaping the future of AI and ML.

---

## Our Platform
### Key Features
- **Models:** Access to over 400,000 pre-trained models, facilitating easy experimentation and deployment.
- **Datasets:** A comprehensive library of 100,000+ datasets for various tasks in computer vision, audio, and NLP.
- **Spaces:** A collaborative environment where users can share applications and discover innovative projects.

### Enterprise Solutions
Tailored for businesses, our enterprise-grade platform offers enhanced security, dedicated support, and access controls that allow teams to implement AI solutions with confidence. 

**Enterprise Pricing:** Starting at **$20/user/month**

---

## Our Community
Hugging Face thrives on community contributions. Our open-source initiatives include:
- **Transformers:** A robust library featuring state-of-the-art ML models for various frameworks (PyTorch, TensorFlow, JAX).
- **Diffusers:** Cutting-edge models for image and audio generation.
- **Tokenizers:** Fast tokenizers for efficient model training and production deployment.

Join our vibrant community on GitHub, Twitter, LinkedIn, and Discord to connect and collaborate with like-minded individuals.

---

## Company Culture
At Hugging Face, we foster a culture of openness and collaboration. Our diverse team values creativity and innovation, and we encourage all members to contribute ideas and participate in the development of our platform. We are passionate about building tools that empower everyone to harness the power of AI and ML.

---

## Career Opportunities
We are always looking for talented individuals who are eager to make an impact in the AI space. Whether you are a developer, researcher, or an enthusiast, we invite you to explore career opportunities at Hugging Face and become part of our mission to democratize machine learning.

**Explore Careers:** [Join us](https://huggingface.co/jobs)

---

## Get In Touch
### Follow Us 
Stay updated with the latest innovations and collaborations happening at Hugging Face:
- **Website:** [huggingface.co](https://huggingface.co)
- **Twitter:** [@HuggingFace](https://twitter.com/huggingface)
- **LinkedIn:** [Hugging Face](https://linkedin.com/company/hugging-face)

---

**Join us in building the future of AI!**

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}



# Hugging Face Brochure

**Company Overview**  
Hugging Face is at the forefront of the AI community, dedicated to building the future of machine learning (ML). Our platform serves as a collaborative space where developers, researchers, and enthusiasts come together to create, discover, and share state-of-the-art models, datasets, and applications.

## Our Products and Services
- **Models**: Access a vast collection of over 400,000 ML models.
- **Datasets**: Browse through 100,000+ datasets designed for various ML tasks.
- **Spaces**: Host and run AI applications effortlessly with our user-friendly environment.
- **Enterprise Solutions**: Tailored for organizations, offering advanced capabilities with enterprise-grade security and support.

## Company Culture
At Hugging Face, we believe in fostering an inclusive and collaborative environment. Our team consists of passionate individuals who are driven by innovation and a shared mission to make machine learning accessible for everyone. We actively encourage the contributions of our community and prioritize open-source development, allowing us to build a strong foundation of tools for ML.

## Our Customers
More than 50,000 organizations trust and use Hugging Face, including industry leaders like Google, Microsoft, Amazon Web Services, and Meta. Our platform empowers companies ranging from startups to enterprises by providing them with the tools they need to innovate in AI. 

## Careers and Opportunities
Join our passionate team and be part of a vibrant community that is shaping the future of AI! We are continuously searching for talented software engineers, data scientists, and other professionals to help us grow. Our hiring philosophy embraces diversity and prioritizes skills, creativity, and experience over traditional credentials.

### Why Work at Hugging Face?
- Collaborate with a world-class team of ML specialists and pioneers.
- Engage in exciting projects that push the boundaries of technology.
- Enjoy a healthy work-life balance and a remote-first work culture.

## Connect with Us
Explore more on our website: [Hugging Face](https://huggingface.co)  
Follow us on social media:  
- [GitHub](https://github.com/huggingface)  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface)  
- [Discord](https://discord.gg/huggingface)  

Join us, and be a part of building the AI community that is shaping our future!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>